In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import wrds
# import data_read
import pull_WRDS_call_reports, pull_treasuries_data
import pull_mbb_data
import data_preprocessing, compute_treasury_changes
# import calc_functions
# import Calc_table_statistic

warnings.filterwarnings("ignore")

In [2]:
# Filename: load_parquet_files.ipynb
# Location: /src

import os
import pandas as pd

# Typically, you would have a cell for imports, settings, etc.

# 1. Identify the folder that holds your Parquet files.
#    Since src/ and data/ are siblings, use a relative path:
data_folder = '../data/manual'


parquet_files = [
    'wrds_call_research.parquet'
]

# 3. Read each Parquet file into a dictionary of DataFrames
all_data = {}
for fname in parquet_files:
    file_path = os.path.join(data_folder, fname)
    
    # Load the parquet file using pandas
    df = pd.read_parquet(file_path)
    
    # Store in the dictionary using the filename as the key
    all_data[fname] = df

# Now you have a dictionary of DataFrames, each one
# corresponding to a Parquet file in data/manual
print("Loaded the following Parquet files:")
for k in all_data:
    print(f"  {k} | {all_data[k].shape} rows/columns")

# You can now work with each DataFrame in `all_data` as needed:
# example:
# display(all_data["lei_main.parquet"].head())

Loaded the following Parquet files:
  wrds_call_research.parquet | (1983357, 371) rows/columns


In [3]:
df = all_data['wrds_call_research.parquet']
df.head()

,rssd9001,rcon9804,date,rssd9050,rssd9055,rssd9048,assets,cash,securities,securities_asu,...,q_intexptradingandborrowed,q_intincpersloans,q_persloansintinc,q_intincreloans,q_intexpdomdep,avgirate_timedep,avgirate_timedep_ytd,avgirate_savdep,avgirate_fordep,fedfundsrate
0,28,54.0,1986-03-31,26606,0,200.0,2779.0,1395.0,0.0,0.0,...,0.0,NaN,NaN,NaN,1.0,None,None,None,None,0.0753
1,28,54.0,1986-06-30,26606,0,200.0,4322.0,153.0,408.0,408.0,...,0.0,NaN,NaN,NaN,26.0,None,None,None,None,0.0689
2,28,54.0,1986-09-30,26606,0,200.0,5118.0,601.0,407.0,407.0,...,0.0,NaN,NaN,NaN,40.0,None,None,None,None,0.0584
3,28,54.0,1986-12-31,26606,0,200.0,5742.0,292.0,611.0,611.0,...,0.0,NaN,NaN,NaN,46.0,None,None,None,None,0.0681
4,28,54.0,1987-03-31,26606,0,200.0,7737.0,701.0,610.0,610.0,...,0.0,NaN,NaN,NaN,61.0,None,0.0566,None,None,0.0610


In [4]:
df.columns

Index(['rssd9001', 'rcon9804', 'date', 'rssd9050', 'rssd9055', 'rssd9048',
       'assets', 'cash', 'securities', 'securities_asu',
       ...
       'q_intexptradingandborrowed', 'q_intincpersloans', 'q_persloansintinc',
       'q_intincreloans', 'q_intexpdomdep', 'avgirate_timedep',
       'avgirate_timedep_ytd', 'avgirate_savdep', 'avgirate_fordep',
       'fedfundsrate'],
      dtype='object', length=371)

In [5]:
import datetime
start_date = datetime.date(2022, 3, 31)
end_date = datetime.date.today()

# Filter rows where the 'date' column is between start_date and end_date
df_filtered = df[
    (df['date'] >= start_date) &
    (df['date'] <= end_date)
]

In [6]:
df_filtered.head()

,rssd9001,rcon9804,date,rssd9050,rssd9055,rssd9048,assets,cash,securities,securities_asu,...,q_intexptradingandborrowed,q_intincpersloans,q_persloansintinc,q_intincreloans,q_intexpdomdep,avgirate_timedep,avgirate_timedep_ytd,avgirate_savdep,avgirate_fordep,fedfundsrate
271,37,59.0,2022-03-31,10057,0,200.0,87842.0,12640.0,53347.0,53347.0,...,NaN,131.0,131.0,157.0,55.0,0.0079,0.0079,0.0017,None,0.0018
273,37,59.0,2022-06-30,10057,0,200.0,84231.0,8061.0,51665.0,51665.0,...,NaN,135.0,135.0,153.0,53.0,0.0076,0.0080,0.0017,None,0.0113
274,37,59.0,2022-09-30,10057,0,200.0,80081.0,4751.0,49439.0,49439.0,...,NaN,141.0,141.0,153.0,53.0,0.0079,0.0081,0.0016,None,0.0252
275,37,59.0,2022-12-31,10057,0,200.0,81497.0,9260.0,50548.0,50548.0,...,NaN,155.0,155.0,154.0,52.0,0.0080,0.0083,0.0017,None,0.0408
277,37,59.0,2023-03-31,10057,0,200.0,80558.0,5977.0,50973.0,50973.0,...,NaN,146.0,146.0,147.0,75.0,0.0117,0.0118,0.0024,None,0.0463


In [7]:
df_filtered['rssd9001'] = df_filtered['rssd9001'].astype(str)

In [8]:
df_filtered['date'] = pd.to_datetime(df_filtered['date'])
# Define required dates
required_dates = [pd.Timestamp("2022-03-31"), pd.Timestamp("2023-03-31")]

# Group by 'rssd9001' and filter out groups that do not contain both required dates
df_filtered = df_filtered.groupby('rssd9001').filter(lambda x: all(date in x['date'].values for date in required_dates))

In [9]:
df_filtered.shape

(24565, 371)

In [10]:
len(df_filtered['rssd9001'].unique())

4913

In [11]:

with open("file.txt", "w") as output:
    output.write(str(list(df_filtered.columns)))


In [12]:
df_filtered['rssd9001'] = df_filtered['rssd9001'].astype(str)

In [13]:
df_new = df_filtered[['rssd9001', 'date', 'assets', 'securitiesheldtomaturity',
                      'securitiesavailableforsale', 'mbsassets', 'absassets', 'loans', 'totaldep', 'alldepuninsured', 'treasurysec', 'timedepuninsured', 'domdepuninsured',
                      'securitiesrmbs_less_3m', 'securitiesrmbs_3m_1y', 'securitiesrmbs_1y_3y', 'securitiesrmbs_3y_5y', 'securitiesrmbs_5y_15y', 'securitiesrmbs_over_15y',
                      'resloans_less_3m', 'resloans_3m_1y', 'resloans_1y_3y', 'resloans_3y_5y', 'resloans_5y_15y', 'resloans_over_15y']]

In [14]:
df_new.head()

,rssd9001,date,assets,securitiesheldtomaturity,securitiesavailableforsale,mbsassets,absassets,loans,totaldep,alldepuninsured,...,securitiesrmbs_1y_3y,securitiesrmbs_3y_5y,securitiesrmbs_5y_15y,securitiesrmbs_over_15y,resloans_less_3m,resloans_3m_1y,resloans_1y_3y,resloans_3y_5y,resloans_5y_15y,resloans_over_15y
271,37,2022-03-31,87842.0,0.0,53347.0,6877.0,0.0,20854.0,73495.0,6162.0,...,0.0,0.0,856.0,6021.0,355.0,354.0,2430.0,2229.0,2641.0,0.0
273,37,2022-06-30,84231.0,0.0,51665.0,6176.0,0.0,22666.0,73039.0,6011.0,...,0.0,0.0,783.0,5392.0,154.0,373.0,2159.0,2105.0,3070.0,0.0
274,37,2022-09-30,80081.0,0.0,49439.0,5776.0,0.0,23521.0,70279.0,3759.0,...,0.0,0.0,727.0,5048.0,202.0,691.0,1782.0,2022.0,3084.0,0.0
275,37,2022-12-31,81497.0,0.0,50548.0,5743.0,0.0,19442.0,70594.0,6543.0,...,0.0,0.0,715.0,5028.0,237.0,589.0,1794.0,2232.0,2831.0,0.0
277,37,2023-03-31,80558.0,0.0,50973.0,5673.0,0.0,21698.0,68896.0,3929.0,...,0.0,0.0,700.0,4974.0,141.0,730.0,1651.0,2600.0,2564.0,0.0


In [15]:
file_path = os.path.join(data_folder, 'ishares_mbs_etf_daily.csv')

# Load the parquet file using pandas
etf_mbs = pd.read_csv(file_path)

In [16]:
etf_mbs.head()

,Date,Close/Last,Volume,Open,High,Low
0,02/25/2025,93.54,2839593,93.43,93.6276,93.060
1,02/24/2025,92.99,2637865,92.64,93.0600,92.550
2,02/21/2025,92.87,1584830,92.52,92.9750,92.415
3,02/20/2025,92.38,1627928,92.26,92.4299,92.250
4,02/19/2025,92.13,2126798,91.91,92.1872,91.870


In [17]:
treasuries_data = pull_treasuries_data.load_from_manual_excel()
treasuries_data.rename(columns=lambda x: x.strip(), inplace=True)
treasuries_data['Effective date'] = pd.to_datetime(treasuries_data['Effective date'])

✅ Successfully loaded Treasury Bond Index data from /Users/aadi/projects/svb-failure-case-study/data/manual/s_&_p_treasury_bond_index.xls


In [18]:
treasuries_data.head()

,Effective date,S&P U.S. Treasury Bond Index
0,2015-01-30,414.51
1,2015-02-02,414.20
2,2015-02-03,412.52
3,2015-02-04,412.79
4,2015-02-05,412.07


In [19]:
def GSIB_bank_id():
    """
    Returns a list of GSIB bank IDs.
    """
    GSIB = [852218, 480228, 476810, 413208, # JP Morgan, Bank of America, Citigroup, HSBC
            2980209, 2182786, 541101, 655839, 1015560, 229913, # Barclays, Goldman Sachs, BNY Mellon, CCB, ICBC, Mizuho
            1456501, 722777, 35301, 925411, 497404, 3212149, # Morgan Stanley, Santander, State Street, Sumitomo Mitsui, TD Bank, UBS
            451965] # Wells Fargo
    return GSIB

In [20]:
df_new['date'] = pd.to_datetime(df_new['date'])
start_date = "03/31/2022"
end_date = "03/31/2023"

In [21]:
etf_mbs_change = etf_mbs.loc[etf_mbs['Date'] == end_date, 'Close/Last'].values[0] / \
                  etf_mbs.loc[etf_mbs['Date'] == start_date, 'Close/Last'].values[0] - 1
start_date = "2022-03-31"
edn_date = "2023-03-31"
treasury_change = treasuries_data.loc[treasuries_data['Effective date'] == end_date, 'S&P U.S. Treasury Bond Index'].values[0] / \
                      treasuries_data.loc[treasuries_data['Effective date'] == start_date, 'S&P U.S. Treasury Bond Index'].values[0] - 1

In [22]:
start_date = "2022-03-31"
edn_date = "2023-03-31"
treasury_change = treasuries_data.loc[treasuries_data['Effective date'] == end_date, 'S&P U.S. Treasury Bond Index'].values[0] / \
                      treasuries_data.loc[treasuries_data['Effective date'] == start_date, 'S&P U.S. Treasury Bond Index'].values[0] - 1

In [ ]:
df_new['mtm_loss_mbs'] = df_new['mbsassets'] * etf_mbs_change
df_new['mtm_loss_treasury'] = df_new['treasurysec'] * treasury_change
df_new['mtm_loss_loans'] = df_new['loans'] * treasury_change

In [24]:
df_new['total_mtm_loss'] = df_new['mtm_loss_mbs'] + df_new['mtm_loss_treasury'] + df_new['mtm_loss_loans']

In [25]:
aggregate_loss = df_new['total_mtm_loss'].sum()

In [26]:
df_new['share_rmbs'] = df_new['mtm_loss_mbs'] / df_new['total_mtm_loss']
df_new['share_treasury_other'] = df_new['mtm_loss_treasury'] / df_new['total_mtm_loss']
df_new['share_residential_mortgage'] = df_new['mtm_loss_loans'] / df_new['total_mtm_loss']

In [27]:
df_new['loss_to_asset'] = df_new['total_mtm_loss'] / df_new['assets']
df_new['uninsured_deposit_mm_asset'] = df_new['alldepuninsured'] / (df_new['assets'] - df_new['total_mtm_loss'])

In [28]:
gsib_list = GSIB_bank_id()
df_new['bank_category'] = np.where(df_new['rssd9001'].isin(gsib_list), "GSIB",
                                   np.where(df_new['assets'] < 1.384e9, "Small", "Large (non-GSIB)"))

In [29]:
df_summary = df_new.groupby('bank_category').agg({
    'total_mtm_loss': 'sum',
    'loss_to_asset': 'mean',
    'uninsured_deposit_mm_asset': 'mean',
    'rssd9001': 'count'
}).rename(columns={'rssd9001': 'number_of_banks'})

In [30]:
df_summary

,total_mtm_loss,loss_to_asset,uninsured_deposit_mm_asset,number_of_banks
bank_category,,,,
Large (non-GSIB),-1.244522e+09,-0.027608,0.485788,29
Small,-2.274063e+09,-0.028930,0.228060,24536


In [31]:
df_assets = df_new[['rssd9001', 'date', 'assets']]

In [32]:
df_assets[df_assets['rssd9001'] == '1016231'].head(10)

,rssd9001,date,assets
287235,1016231,2022-03-31,1398582.0
287236,1016231,2022-06-30,1448805.0
287237,1016231,2022-09-30,1566467.0
287238,1016231,2022-12-31,1599507.0
287239,1016231,2023-03-31,1697313.0


In [33]:
df_insured = df_new[['rssd9001', 'date', 'alldepuninsured','timedepuninsured', 'domdepuninsured']]

In [34]:
df_insured[df_insured['rssd9001'] == '37640'].tail()

,rssd9001,date,alldepuninsured,timedepuninsured,domdepuninsured
68501,37640,2022-03-31,2396616.0,71800.0,2396616.0
68502,37640,2022-06-30,2348762.0,58467.0,2348762.0
68503,37640,2022-09-30,2262791.0,62276.0,2262791.0
68504,37640,2022-12-31,2143959.0,65081.0,2143959.0
68505,37640,2023-03-31,1957134.0,121494.0,1957134.0


In [35]:
def GSIB_bank_id():
    """
    Returns a list of GSIB bank IDs.
    """
    #GSIB = [35301,93619,229913,398668,413208,451965,476810,480228,488318,
     #497404,541101,651448,688079,722777,812164,852218,934329,1225761,
     #1443266,1456501,2182786,2362458,2489805,2531991,3066025]
    GSIB = [852218, 480228, 476810, 413208, #JP Morgan, Bank of America, Citigroup, HSBC
      2980209, 2182786, 541101, 655839, 1015560, 229913,#Barclays, Goldman Sachs, BNY Mellon, CCB COMMUNITY BANK, ICBC, Mizuho
       1456501, 722777, 35301, 925411, 497404, 3212149, #Morgan Stanley, Santander, State Street, Sumitomo Mitsui, TD Bank, UBS
      451965] #wells fargo
    return GSIB

In [36]:
df_assets.head()

,rssd9001,date,assets
271,37,2022-03-31,87842.0
273,37,2022-06-30,84231.0
274,37,2022-09-30,80081.0
275,37,2022-12-31,81497.0
277,37,2023-03-31,80558.0


In [37]:
df_asset = df_assets #total assets all banks
#GSIB Banks
GSIB = GSIB_bank_id() #list of GSIB bank IDs
GSIB = [str(element) for element in GSIB]
df_asset_GSIB = df_asset[df_asset['rssd9001'].isin(GSIB)] #total assets all GSIB banks
#Large non-GSIB Banks
df_asset_large_ex_GSIB = df_asset[(~df_asset['rssd9001'].isin(GSIB)) & (df_asset['assets']>1384000)] #total assets all large non-GSIB banks
df_asset_small = df_asset[(~df_asset['rssd9001'].isin(GSIB)) & (df_asset['assets']<=1384000)] 

In [38]:
df_asset_GSIB.head()

,rssd9001,date,assets
63654,35301,2022-03-31,318494000.0
63655,35301,2022-06-30,296434000.0
63656,35301,2022-09-30,300010000.0
63657,35301,2022-12-31,298020000.0
63658,35301,2023-03-31,287069000.0


In [39]:
df_rmbs = df_new[['rssd9001', 'date', 'securitiesrmbs_less_3m', 'securitiesrmbs_3m_1y', 'securitiesrmbs_1y_3y', 'securitiesrmbs_3y_5y', 'securitiesrmbs_5y_15y', 'securitiesrmbs_over_15y']]

In [40]:
df_rmbs.head()

,rssd9001,date,securitiesrmbs_less_3m,securitiesrmbs_3m_1y,securitiesrmbs_1y_3y,securitiesrmbs_3y_5y,securitiesrmbs_5y_15y,securitiesrmbs_over_15y
271,37,2022-03-31,0.0,0.0,0.0,0.0,856.0,6021.0
273,37,2022-06-30,0.0,0.0,0.0,0.0,783.0,5392.0
274,37,2022-09-30,0.0,0.0,0.0,0.0,727.0,5048.0
275,37,2022-12-31,0.0,0.0,0.0,0.0,715.0,5028.0
277,37,2023-03-31,0.0,0.0,0.0,0.0,700.0,4974.0


In [41]:
df_loans = df_new[['rssd9001', 'date', 'resloans_less_3m', 'resloans_3m_1y', 'resloans_1y_3y', 'resloans_3y_5y', 'resloans_5y_15y', 'resloans_over_15y']]

In [42]:
df_loans.head()

,rssd9001,date,resloans_less_3m,resloans_3m_1y,resloans_1y_3y,resloans_3y_5y,resloans_5y_15y,resloans_over_15y
271,37,2022-03-31,355.0,354.0,2430.0,2229.0,2641.0,0.0
273,37,2022-06-30,154.0,373.0,2159.0,2105.0,3070.0,0.0
274,37,2022-09-30,202.0,691.0,1782.0,2022.0,3084.0,0.0
275,37,2022-12-31,237.0,589.0,1794.0,2232.0,2831.0,0.0
277,37,2023-03-31,141.0,730.0,1651.0,2600.0,2564.0,0.0


In [43]:
df_treasuries = df_filtered[['rssd9001', 'date', 'securitiestreasury_less_3m',
'securitiestreasury_3m_1y',
'securitiestreasury_1y_3y',
'securitiestreasury_3y_5y',
'securitiestreasury_5y_15y',
'securitiestreasury_over_15y']]

In [44]:
df_treasuries.head()

,rssd9001,date,securitiestreasury_less_3m,securitiestreasury_3m_1y,securitiestreasury_1y_3y,securitiestreasury_3y_5y,securitiestreasury_5y_15y,securitiestreasury_over_15y
271,37,2022-03-31,0.0,0.0,330.0,708.0,26220.0,19212.0
273,37,2022-06-30,0.0,1311.0,701.0,25997.0,0.0,17481.0
274,37,2022-09-30,0.0,0.0,1289.0,694.0,27195.0,14486.0
275,37,2022-12-31,0.0,326.0,1361.0,300.0,28701.0,14116.0
277,37,2023-03-31,0.0,817.0,2362.0,536.0,28421.0,13165.0


In [45]:
df_other_loans = df_filtered[['rssd9001', 'date', 'loansleases_less_3m',
'loansleases_3m_1y',
'loansleases_1y_3y',
'loansleases_3y_5y',
'loansleases_5y_15y',
'loansleases_over_15y']]
df_other_loans.head()

,rssd9001,date,loansleases_less_3m,loansleases_3m_1y,loansleases_1y_3y,loansleases_3y_5y,loansleases_5y_15y,loansleases_over_15y
271,37,2022-03-31,1437.0,3928.0,5358.0,5317.0,4747.0,0.0
273,37,2022-06-30,536.0,6539.0,4884.0,5228.0,5370.0,0.0
274,37,2022-09-30,5065.0,3502.0,4598.0,4967.0,5330.0,0.0
275,37,2022-12-31,715.0,3154.0,4901.0,6211.0,4402.0,0.0
277,37,2023-03-31,1799.0,3552.0,5370.0,6795.0,4119.0,0.0


In [46]:
rcon_series_1 = pull_WRDS_call_reports.load_RCON_series_1()

In [47]:
rcon_series_1['rssd9999'] = pd.to_datetime(rcon_series_1['rssd9999'])
# Define required dates
required_dates = [pd.Timestamp("2022-03-31"), pd.Timestamp("2023-03-31")]

rcon_series_1 = rcon_series_1.groupby('rssd9001').filter(lambda x: all(date in x['rssd9999'].values for date in required_dates))

In [48]:
rcon_series_1 = rcon_series_1.rename(columns={
    'rssd9999': 'date'
})

In [49]:
rcon_series_1.head()

,rssd9001,rssd9017,date,uninsured_deposits,insured_deposit_1,insured_deposit_2
0,23643,COMMERCIAL BANK & TRUST COMPANY,2022-09-30,NaN,113347.0,5371.0
1,37,BANK OF HANCOCK COUNTY,2021-12-31,NaN,56295.0,2430.0
2,37,BANK OF HANCOCK COUNTY,2022-03-31,NaN,57430.0,2428.0
3,37,BANK OF HANCOCK COUNTY,2022-06-30,NaN,57947.0,2352.0
4,37,BANK OF HANCOCK COUNTY,2022-09-30,NaN,57469.0,2324.0


In [50]:
df_uninsured = rcon_series_1[['rssd9001', 'date', 'uninsured_deposits']]

In [51]:
df_insured = rcon_series_1[['rssd9001', 'date', 'insured_deposit_1', 'insured_deposit_2']]

In [52]:
asset_small = df_asset_small['rssd9001'].unique()
asset_large_ex_GSIB = df_asset_large_ex_GSIB['rssd9001'].unique()

In [53]:
len(df_asset_large_ex_GSIB), len(df_asset_small), len(GSIB)

(4236, 20235, 17)

In [54]:
def get_partitioned_data(df, gsib, large, small):
    df_gsib = df[df['rssd9001'].isin(gsib)]
    df_large = df[df['rssd9001'].isin(large)]
    df_small = df[df['rssd9001'].isin(small)]
    return df_gsib, df_large, df_small

In [55]:
df_rmbs_gsib, df_rmbs_large, df_rmbs_small = get_partitioned_data(df_rmbs, GSIB, asset_large_ex_GSIB, asset_small)

df_treasuries_gsib, df_treasuries_large, df_treasuries_small = get_partitioned_data(df_treasuries, GSIB, asset_large_ex_GSIB, asset_small)

df_loans_gsib, df_loans_large, df_loans_small = get_partitioned_data(df_loans, GSIB, asset_large_ex_GSIB, asset_small)

df_other_loans_gsib, df_other_loans_large, df_other_loans_small = get_partitioned_data(df_other_loans, GSIB, asset_large_ex_GSIB, asset_small)

df_insured_gsib, df_insured_large, df_insured_small = get_partitioned_data(df_insured, GSIB, asset_large_ex_GSIB, asset_small)

df_uninsured_gsib, df_uninsured_large, df_uninsured_small = get_partitioned_data(df_uninsured, GSIB, asset_large_ex_GSIB, asset_small)

In [56]:
etf_mbs_change / treasury_change

1.8904405040465027

In [57]:
start_date = "03/31/2022"
end_date = "03/31/2023"
etf_mbs_change = (etf_mbs.loc[etf_mbs['Date'] == end_date, 'Close/Last'].values[0] /
                  etf_mbs.loc[etf_mbs['Date'] == start_date, 'Close/Last'].values[0]) - 1
start_date = "2022-03-31"
edn_date = "2023-03-31"
treasury_change = (treasuries_data.loc[treasuries_data['Effective date'] == end_date, 'S&P U.S. Treasury Bond Index'].values[0] /
                      treasuries_data.loc[treasuries_data['Effective date'] == start_date, 'S&P U.S. Treasury Bond Index'].values[0]) - 1

In [58]:
rmbs_multiplier = etf_mbs_change / treasury_change

In [59]:
# df_asset, df_rmbs, df_treasuries, df_loans, df_other_loans, 

In [60]:
treasury_prices = pd.read_excel("../data/manual/combined_index_df.xlsx")

In [61]:
#treasury_prices['date'] = treasury_prices['date'].astype(str)
treasury_prices.set_index('date', inplace=True)

In [62]:
start_date = "2022-03-31"
end_date ="2023-03-31"
price_change = {
    '<1y': treasury_prices.loc[end_date, 'iShares 0-1'] / treasury_prices.loc[start_date, 'iShares 0-1'] - 1,
    '1y-3y': treasury_prices.loc[end_date, 'iShares 1-3'] / treasury_prices.loc[start_date, 'iShares 1-3'] - 1,
    '3y-5y': treasury_prices.loc[end_date, 'sp 3-5'] / treasury_prices.loc[start_date, 'sp 3-5'] - 1,
    '7y-10y': 0.5 * (treasury_prices.loc[end_date, 'iShares 7-10'] / treasury_prices.loc[start_date, 'iShares 7-10'] - 1) + 0.5 * (treasury_prices.loc[end_date, 'iShares 10-20'] / treasury_prices.loc[start_date, 'iShares 10-20'] - 1),
    '>20y': treasury_prices.loc[end_date, 'iShares 20+'] / treasury_prices.loc[start_date, 'iShares 20+'] - 1,
}
price_change

{'<1y': 0.022747325653670325,
 '1y-3y': 0.001782208578348543,
 '3y-5y': -0.012701806906874835,
 '7y-10y': -0.09156480703416248,
 '>20y': -0.17378695410977818}

In [63]:
# Define the mapping of buckets to be used for aggregation
bucket_mapping = {
    '<3m': '<1y',
    '3m-1y': '<1y',
    '1y-3y': '1y-3y',
    '3y-5y': '3y-5y',
    '5y-15y': '7y-10y',  # Assuming '5y-15y' should be mapped to '7y-10y' based on provided price_change calculation
    '>15y': '>20y',
}

In [64]:
df_asset, df_rmbs, df_treasuries, df_loans, df_other_loans, 

(       rssd9001       date   assets
 271          37 2022-03-31  87842.0
 273          37 2022-06-30  84231.0
 274          37 2022-09-30  80081.0
 275          37 2022-12-31  81497.0
 277          37 2023-03-31  80558.0
 ...         ...        ...      ...
 483312  5691319 2022-03-31  19216.0
 483313  5691319 2022-06-30  24316.0
 483314  5691319 2022-09-30  28422.0
 483315  5691319 2022-12-31  35395.0
 483316  5691319 2023-03-31  36662.0
 
 [24565 rows x 3 columns],
        rssd9001       date  securitiesrmbs_less_3m  securitiesrmbs_3m_1y  \
 271          37 2022-03-31                     0.0                   0.0   
 273          37 2022-06-30                     0.0                   0.0   
 274          37 2022-09-30                     0.0                   0.0   
 275          37 2022-12-31                     0.0                   0.0   
 277          37 2023-03-31                     0.0                   0.0   
 ...         ...        ...                     ...              

In [65]:
df_rmbs = df_rmbs.rename(columns={
'securitiesrmbs_less_3m': '<3m',
'securitiesrmbs_3m_1y':'3m-1y',
'securitiesrmbs_1y_3y':'1y-3y',
'securitiesrmbs_3y_5y':'3y-5y',
'securitiesrmbs_5y_15y':'5y-15y',
'securitiesrmbs_over_15y':'>15y',
})

In [66]:
df_treasuries.columns

Index(['rssd9001', 'date', 'securitiestreasury_less_3m',
       'securitiestreasury_3m_1y', 'securitiestreasury_1y_3y',
       'securitiestreasury_3y_5y', 'securitiestreasury_5y_15y',
       'securitiestreasury_over_15y'],
      dtype='object')

In [67]:
df_treasuries = df_treasuries.rename(columns={
'securitiestreasury_less_3m': '<3m',
'securitiestreasury_3m_1y':'3m-1y',
'securitiestreasury_1y_3y':'1y-3y',
'securitiestreasury_3y_5y':'3y-5y',
'securitiestreasury_5y_15y':'5y-15y',
'securitiestreasury_over_15y':'>15y',
})

In [68]:
df_loans.columns

Index(['rssd9001', 'date', 'resloans_less_3m', 'resloans_3m_1y',
       'resloans_1y_3y', 'resloans_3y_5y', 'resloans_5y_15y',
       'resloans_over_15y'],
      dtype='object')

In [69]:
df_loans = df_loans.rename(columns={
'resloans_less_3m': '<3m',
'resloans_3m_1y':'3m-1y',
'resloans_1y_3y':'1y-3y',
'resloans_3y_5y':'3y-5y',
'resloans_5y_15y':'5y-15y',
'resloans_over_15y':'>15y',
})

In [70]:
df_other_loans.columns

Index(['rssd9001', 'date', 'loansleases_less_3m', 'loansleases_3m_1y',
       'loansleases_1y_3y', 'loansleases_3y_5y', 'loansleases_5y_15y',
       'loansleases_over_15y'],
      dtype='object')

In [71]:
df_other_loans = df_other_loans.rename(columns={
'loansleases_less_3m': '<3m',
'loansleases_3m_1y':'3m-1y',
'loansleases_1y_3y':'1y-3y',
'loansleases_3y_5y':'3y-5y',
'loansleases_5y_15y':'5y-15y',
'loansleases_over_15y':'>15y',
})

In [72]:
df_rmbs_gsib, df_rmbs_large, df_rmbs_small = get_partitioned_data(df_rmbs, GSIB, asset_large_ex_GSIB, asset_small)

df_treasuries_gsib, df_treasuries_large, df_treasuries_small = get_partitioned_data(df_treasuries, GSIB, asset_large_ex_GSIB, asset_small)

df_loans_gsib, df_loans_large, df_loans_small = get_partitioned_data(df_loans, GSIB, asset_large_ex_GSIB, asset_small)

df_other_loans_gsib, df_other_loans_large, df_other_loans_small = get_partitioned_data(df_other_loans, GSIB, asset_large_ex_GSIB, asset_small)

df_insured_gsib, df_insured_large, df_insured_small = get_partitioned_data(df_insured, GSIB, asset_large_ex_GSIB, asset_small)

df_uninsured_gsib, df_uninsured_large, df_uninsured_small = get_partitioned_data(df_uninsured, GSIB, asset_large_ex_GSIB, asset_small)

In [73]:
# Aggregate the assets for each bank
aggregated_assets = {}
for name, df in zip(['RMBS', 'Loans', 'Treasury', 'OtherLoan'], 
                    [df_rmbs, df_loans, df_treasuries, df_other_loans]):
    # Ensure columns for aggregation are present
    columns_to_aggregate = [col for col in list(bucket_mapping.keys()) if col in df.columns]
    aggregated_assets[name] = df.groupby(['rssd9001', 'date'])[columns_to_aggregate].sum().reset_index()

# Initialize DataFrame to store results
bank_losses_assets = pd.DataFrame(columns=[
    'date', 'bank_ID', 'RMBs_loss', 'treasury_loss', 'loans_loss', 'other_loan_loss', 
    'total_loss', 'Share RMBs', 'Share Treasury and Other', 
    'Share Residential Mortgage', 'Share Other Loan', 'RMBs_asset', 'treasury_asset', 
    'residential_mortgage_asset', 'other_loan_asset', 'core_asset', 'gross_asset', 'loss/core_asset', 'loss/gross_asset',
])

In [74]:
aggregated_assets['Loans']

,rssd9001,date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
0,1000052,2022-03-31,3534.0,6017.0,10089.0,4826.0,3743.0,4574.0
1,1000052,2022-06-30,4439.0,4286.0,10963.0,7732.0,3618.0,5110.0
2,1000052,2022-09-30,2728.0,6188.0,12085.0,6792.0,3439.0,5158.0
3,1000052,2022-12-31,1507.0,5696.0,12074.0,11138.0,3412.0,5490.0
4,1000052,2023-03-31,2361.0,4777.0,12237.0,14714.0,4008.0,5567.0
...,...,...,...,...,...,...,...,...
24560,999935,2022-03-31,0.0,1636.0,2941.0,1735.0,22252.0,100473.0
24561,999935,2022-06-30,1488.0,2723.0,890.0,2039.0,21722.0,110097.0
24562,999935,2022-09-30,1084.0,4598.0,449.0,1884.0,20395.0,114353.0
24563,999935,2022-12-31,1254.0,5647.0,1015.0,1187.0,18295.0,119752.0


In [75]:
df_asset.columns

Index(['rssd9001', 'date', 'assets'], dtype='object')

In [76]:
df_asset.head()

,rssd9001,date,assets
271,37,2022-03-31,87842.0
273,37,2022-06-30,84231.0
274,37,2022-09-30,80081.0
275,37,2022-12-31,81497.0
277,37,2023-03-31,80558.0


In [77]:
# Iterate over each bank to calculate losses and assets
for _, df_row in df_asset.iterrows():
    bank_id = df_row['rssd9001']
    bank_date = df_row['date']
    bank_total_asset = df_row['assets']
    
    #Initialize variables for loss and asset calculations
    rmbs_loss = loans_loss = treasury_loss = other_loan_loss = total_loss = 0
    rmbs_asset = treasury_asset = loan_asset = other_loan_asset = core_asset = 0
    
    #Calculating losses for RMBs
    if 'RMBS' in aggregated_assets and not aggregated_assets['RMBS'].empty:
        rmbs_row = aggregated_assets['RMBS'][(aggregated_assets['RMBS']['date'] == bank_date) & (aggregated_assets['RMBS']['rssd9001'] == bank_id)]
        for bucket, treasury_bucket in bucket_mapping.items():
            if bucket in rmbs_row.columns:
                asset_amount = rmbs_row.iloc[0][bucket] if not rmbs_row.empty else 0
                rmbs_loss += (asset_amount * rmbs_multiplier * price_change[treasury_bucket])
                rmbs_asset += asset_amount

    #Calculating losses for loans
    loans_row = aggregated_assets['Loans'][(aggregated_assets['Loans']['date'] == bank_date) &(aggregated_assets['Loans']['rssd9001'] == bank_id)]
    if not loans_row.empty:
        for bucket, treasury_bucket in bucket_mapping.items():
            if bucket in loans_row.columns:
                asset_amount = loans_row.iloc[0][bucket]
                loans_loss += (asset_amount * rmbs_multiplier * price_change[treasury_bucket])
                loan_asset += asset_amount

    #Calculating Treasuries
    treasury_row = aggregated_assets['Treasury'][(aggregated_assets['Treasury']['date'] == bank_date) & (aggregated_assets['Treasury']['rssd9001'] == bank_id)]
    if not treasury_row.empty:
        for bucket, treasury_bucket in bucket_mapping.items():
            if bucket in treasury_row.columns:
                asset_amount = treasury_row.iloc[0][bucket]
                treasury_loss += (asset_amount * price_change[treasury_bucket])
                treasury_asset += asset_amount
    #Other loans
    other_loan_row = aggregated_assets['OtherLoan'][(aggregated_assets['OtherLoan']['date'] == bank_date) & (aggregated_assets['OtherLoan']['rssd9001'] == bank_id)]
    if not other_loan_row.empty:
        for bucket, treasury_bucket in bucket_mapping.items():
            if bucket in other_loan_row.columns:
                asset_amount = other_loan_row.iloc[0][bucket]
                other_loan_loss += (asset_amount * price_change[treasury_bucket])
                other_loan_asset += asset_amount

        # Calculate total loss and core asset      
    total_loss = rmbs_loss + treasury_loss + loans_loss + other_loan_loss
    core_asset = rmbs_asset + treasury_asset + loan_asset + other_loan_asset

    # Append the results to the DataFrame
    bank_losses_assets.loc[len(bank_losses_assets)] = {
        'date': bank_date,
        'bank_ID': bank_id,
        'RMBs_loss': rmbs_loss,
        'treasury_loss': treasury_loss,
        'loans_loss': loans_loss,
        'other_loan_loss': other_loan_loss,
        'total_loss': total_loss,
        'Share RMBs': rmbs_loss / total_loss if total_loss else 0,
        'Share Treasury and Other': treasury_loss / total_loss if total_loss else 0,
        'Share Residential Mortgage': loans_loss / total_loss if total_loss else 0,
        'Share Other Loan': other_loan_loss / total_loss if total_loss else 0,
        'RMBs_asset': rmbs_asset,
        'treasury_asset': treasury_asset,
        'residential_mortgage_asset': loan_asset,
        'other_loan_asset': other_loan_asset,
        'core_asset': core_asset,
        'gross_asset': bank_total_asset,
        'loss/core_asset': -(total_loss / core_asset) if core_asset else 0,
        'loss/gross_asset': -(total_loss / bank_total_asset) if bank_total_asset else 0,
    }

In [78]:
bank_losses_assets.head()

,date,bank_ID,RMBs_loss,treasury_loss,loans_loss,other_loan_loss,total_loss,Share RMBs,Share Treasury and Other,Share Residential Mortgage,Share Other Loan,RMBs_asset,treasury_asset,residential_mortgage_asset,other_loan_asset,core_asset,gross_asset,loss/core_asset,loss/gross_asset
0,2022-03-31,37,-2126.274328,-5748.028953,-471.998308,-370.605171,-8716.906760,0.243925,0.659412,0.054147,0.042516,6877.0,46470.0,8009.0,20787.0,82143.0,87842.0,0.106119,0.099234
1,2022-06-30,37,-1906.990366,-3337.107547,-552.019275,-388.466425,-6184.583612,0.308346,0.539585,0.089257,0.062812,6175.0,45490.0,7861.0,22557.0,82083.0,84231.0,0.075345,0.073424
2,2022-09-30,37,-1784.281228,-5014.100532,-537.980918,-348.059362,-7684.422040,0.232195,0.652502,0.070009,0.045294,5775.0,43664.0,7781.0,23462.0,80682.0,80081.0,0.095243,0.095958
3,2022-12-31,37,-1775.633376,-5075.147499,-502.070427,-385.215196,-7738.066498,0.229467,0.655868,0.064883,0.049782,5743.0,44804.0,7683.0,19383.0,77613.0,81497.0,0.099701,0.094949
4,2023-03-31,37,-1755.296078,-4874.282658,-463.236407,-332.172818,-7424.987962,0.236404,0.656470,0.062389,0.044737,5674.0,45301.0,7686.0,21635.0,80296.0,80558.0,0.092470,0.092169


In [79]:
bank_count = len(bank_losses_assets.index)
def get_final_stats(bank_losses_assets):
    bank_count = len(bank_losses_assets.index)
    final_stats = pd.DataFrame({
        'Aggregate Loss': [f"{-round(bank_losses_assets['total_loss'].sum() / 1e9, 1)}T"],  # Convert to trillions
        'Bank Level Loss': [f"{-round(bank_losses_assets['total_loss'].median() / 1e3, 1)}M"],  # Convert to millions
        'Bank Level Loss Std': [f"{round(bank_losses_assets['total_loss'].std() / 1e6, 2)}B"],  # Std deviation for Bank Level Loss
        'Share RMBS': [round(bank_losses_assets['Share RMBs'].median() * 100, 1)],  # Median percentage
        'Share RMBS Std': [round(bank_losses_assets['Share RMBs'].std() * 100, 1)],  # Std deviation for Share RMBS
        'Share Treasury and Other': [round(bank_losses_assets['Share Treasury and Other'].median() * 100, 1)],  # Median percentage
        'Share Treasury and Other Std': [round(bank_losses_assets['Share Treasury and Other'].std() * 100, 1)],  # Std deviation
        'Share Residential Mortgage': [round(bank_losses_assets['Share Residential Mortgage'].median() * 100, 1)],  # Median percentage
        'Share Residential Mortgage Std': [round(bank_losses_assets['Share Residential Mortgage'].std() * 100, 1)],  # Std deviation
        'Share Other Loan': [round(bank_losses_assets['Share Other Loan'].median() * 100, 1)],  # Median percentage
        'Share Other Loan Std': [round(bank_losses_assets['Share Other Loan'].std() * 100, 1)],  # Std deviation
        'Loss/Asset': [round(bank_losses_assets['loss/gross_asset'].median() * 100, 1)],  # Median percentage
        'Loss/Asset Std': [round(bank_losses_assets['loss/gross_asset'].std() * 100, 1)],  # Std deviation
        'Number of Banks': [len(bank_losses_assets.index.unique())]  # Count of unique banks
    })
    return final_stats

In [80]:
bank_losses_assets_1 = bank_losses_assets
bank_losses_assets_1['date'] = pd.to_datetime(bank_losses_assets_1['date'])
bank_losses_assets_1 = bank_losses_assets_1[bank_losses_assets_1['date'] == pd.Timestamp('2022-03-31')]
final_stats_1 = get_final_stats(bank_losses_assets_1)

In [81]:
final_stats_1.T

,0
Aggregate Loss,1.6T
Bank Level Loss,15.0M
Bank Level Loss Std,5.57B
Share RMBS,9.4
Share RMBS Std,83.9
Share Treasury and Other,10.0
Share Treasury and Other Std,105.0
Share Residential Mortgage,20.9
Share Residential Mortgage Std,46.0
Share Other Loan,33.5


In [82]:
bank_losses_assets_2 = bank_losses_assets
bank_losses_assets_2['date'] = pd.to_datetime(bank_losses_assets_2['date'])
bank_losses_assets_2 = bank_losses_assets_2[bank_losses_assets_2['date'] == pd.Timestamp('2023-03-31')]
final_stats_2 = get_final_stats(bank_losses_assets_2)

In [83]:
final_stats_2.T

,0
Aggregate Loss,1.6T
Bank Level Loss,14.8M
Bank Level Loss Std,5.33B
Share RMBS,9.0
Share RMBS Std,59.1
Share Treasury and Other,8.1
Share Treasury and Other Std,511.7
Share Residential Mortgage,24.3
Share Residential Mortgage Std,100.7
Share Other Loan,34.5


In [84]:
def calculate_losses(df_rmbs, df_loans, df_treasuries, df_other_loans):
    aggregated_assets = {}
    for name, df in zip(['RMBS', 'Loans', 'Treasury', 'OtherLoan'], 
                        [df_rmbs, df_loans, df_treasuries, df_other_loans]):
        # Ensure columns for aggregation are present
        columns_to_aggregate = [col for col in list(bucket_mapping.keys()) if col in df.columns]
        aggregated_assets[name] = df.groupby(['rssd9001', 'date'])[columns_to_aggregate].sum().reset_index()

    # Initialize DataFrame to store results
    bank_losses_assets = pd.DataFrame(columns=[
        'date', 'bank_ID', 'RMBs_loss', 'treasury_loss', 'loans_loss', 'other_loan_loss', 
        'total_loss', 'Share RMBs', 'Share Treasury and Other', 
        'Share Residential Mortgage', 'Share Other Loan', 'RMBs_asset', 'treasury_asset', 
        'residential_mortgage_asset', 'other_loan_asset', 'core_asset', 'gross_asset', 'loss/core_asset', 'loss/gross_asset',
    ])

    # Iterate over each bank to calculate losses and assets
    for _, df_row in df_asset.iterrows():
        bank_id = df_row['rssd9001']
        bank_date = df_row['date']
        bank_total_asset = df_row['assets']
        
        #Initialize variables for loss and asset calculations
        rmbs_loss = loans_loss = treasury_loss = other_loan_loss = total_loss = 0
        rmbs_asset = treasury_asset = loan_asset = other_loan_asset = core_asset = 0
        
        #Calculating losses for RMBs
        if 'RMBS' in aggregated_assets and not aggregated_assets['RMBS'].empty:
            rmbs_row = aggregated_assets['RMBS'][(aggregated_assets['RMBS']['date'] == bank_date) & (aggregated_assets['RMBS']['rssd9001'] == bank_id)]
            for bucket, treasury_bucket in bucket_mapping.items():
                if bucket in rmbs_row.columns:
                    asset_amount = rmbs_row.iloc[0][bucket] if not rmbs_row.empty else 0
                    rmbs_loss += (asset_amount * rmbs_multiplier * price_change[treasury_bucket])
                    rmbs_asset += asset_amount

        #Calculating losses for loans
        loans_row = aggregated_assets['Loans'][(aggregated_assets['Loans']['date'] == bank_date) &(aggregated_assets['Loans']['rssd9001'] == bank_id)]
        if not loans_row.empty:
            for bucket, treasury_bucket in bucket_mapping.items():
                if bucket in loans_row.columns:
                    asset_amount = loans_row.iloc[0][bucket]
                    loans_loss += (asset_amount * rmbs_multiplier * price_change[treasury_bucket])
                    loan_asset += asset_amount

        #Calculating Treasuries
        treasury_row = aggregated_assets['Treasury'][(aggregated_assets['Treasury']['date'] == bank_date) & (aggregated_assets['Treasury']['rssd9001'] == bank_id)]
        if not treasury_row.empty:
            for bucket, treasury_bucket in bucket_mapping.items():
                if bucket in treasury_row.columns:
                    asset_amount = treasury_row.iloc[0][bucket]
                    treasury_loss += (asset_amount * price_change[treasury_bucket])
                    treasury_asset += asset_amount
        #Other loans
        other_loan_row = aggregated_assets['OtherLoan'][(aggregated_assets['OtherLoan']['date'] == bank_date) & (aggregated_assets['OtherLoan']['rssd9001'] == bank_id)]
        if not other_loan_row.empty:
            for bucket, treasury_bucket in bucket_mapping.items():
                if bucket in other_loan_row.columns:
                    asset_amount = other_loan_row.iloc[0][bucket]
                    other_loan_loss += (asset_amount * price_change[treasury_bucket])
                    other_loan_asset += asset_amount

            # Calculate total loss and core asset      
        total_loss = rmbs_loss + treasury_loss + loans_loss + other_loan_loss
        core_asset = rmbs_asset + treasury_asset + loan_asset + other_loan_asset

        # Append the results to the DataFrame
        bank_losses_assets.loc[len(bank_losses_assets)] = {
            'date': bank_date,
            'bank_ID': bank_id,
            'RMBs_loss': rmbs_loss,
            'treasury_loss': treasury_loss,
            'loans_loss': loans_loss,
            'other_loan_loss': other_loan_loss,
            'total_loss': total_loss,
            'Share RMBs': rmbs_loss / total_loss if total_loss else 0,
            'Share Treasury and Other': treasury_loss / total_loss if total_loss else 0,
            'Share Residential Mortgage': loans_loss / total_loss if total_loss else 0,
            'Share Other Loan': other_loan_loss / total_loss if total_loss else 0,
            'RMBs_asset': rmbs_asset,
            'treasury_asset': treasury_asset,
            'residential_mortgage_asset': loan_asset,
            'other_loan_asset': other_loan_asset,
            'core_asset': core_asset,
            'gross_asset': bank_total_asset,
            'loss/core_asset': -(total_loss / core_asset) if core_asset else 0,
            'loss/gross_asset': -(total_loss / bank_total_asset) if bank_total_asset else 0,
        }
    return bank_losses_assets

In [85]:
bank_losses_assets_gsib = calculate_losses(df_loans=df_loans_gsib, df_rmbs=df_rmbs_gsib, df_treasuries=df_treasuries_gsib, df_other_loans=df_other_loans_gsib)


bank_losses_assets_1 = bank_losses_assets_gsib
bank_losses_assets_1['date'] = pd.to_datetime(bank_losses_assets_1['date'])
bank_losses_assets_1 = bank_losses_assets_1[bank_losses_assets_1['date'] == pd.Timestamp('2022-03-31')]
final_stats_1 = get_final_stats(bank_losses_assets_1)

In [86]:
final_stats_1.T

,0
Aggregate Loss,0.7T
Bank Level Loss,-0.0M
Bank Level Loss Std,5.33B
Share RMBS,0.0
Share RMBS Std,3.1
Share Treasury and Other,0.0
Share Treasury and Other Std,0.7
Share Residential Mortgage,0.0
Share Residential Mortgage Std,2.2
Share Other Loan,0.0


In [87]:
bank_losses_assets_1 = bank_losses_assets_gsib
bank_losses_assets_1['date'] = pd.to_datetime(bank_losses_assets_1['date'])
bank_losses_assets_1 = bank_losses_assets_1[bank_losses_assets_1['date'] == pd.Timestamp('2023-03-31')]
final_stats_2 = get_final_stats(bank_losses_assets_1)

In [88]:
final_stats_2.T

,0
Aggregate Loss,0.7T
Bank Level Loss,-0.0M
Bank Level Loss Std,5.06B
Share RMBS,0.0
Share RMBS Std,3.1
Share Treasury and Other,0.0
Share Treasury and Other Std,0.5
Share Residential Mortgage,0.0
Share Residential Mortgage Std,2.9
Share Other Loan,0.0


In [89]:
bank_losses_assets_small = calculate_losses(df_loans=df_loans_small, df_rmbs=df_rmbs_small, df_treasuries=df_treasuries_small, df_other_loans=df_other_loans_small)


bank_losses_assets_1 = bank_losses_assets_small
bank_losses_assets_1['date'] = pd.to_datetime(bank_losses_assets_1['date'])
bank_losses_assets_1 = bank_losses_assets_1[bank_losses_assets_1['date'] == pd.Timestamp('2022-03-31')]
final_stats_1 = get_final_stats(bank_losses_assets_1)
final_stats_1.T

,0
Aggregate Loss,0.1T
Bank Level Loss,7.8M
Bank Level Loss Std,0.03B
Share RMBS,3.3
Share RMBS Std,81.7
Share Treasury and Other,6.9
Share Treasury and Other Std,104.7
Share Residential Mortgage,14.6
Share Residential Mortgage Std,44.7
Share Other Loan,29.0
